# Lesson 2: Explore Carbon Intensity on the Grid

* In this classroom, the libraries have been already installed for you.
* If you would like to run this code on your own machine, make sure to get your Electricity Maps API from their [website](https://www.electricitymaps.com/free-tier-api).

#### Load the Electricity Maps API for this notebook

In [1]:
from helper import load_env
load_env()

### Electricity Map's live carbon intensity  endpoint

* Set location coordinates.

Remember, you can get the coordinates from [Google Maps](https://www.google.com/maps).

In [2]:
# Create a dictionary with the coordinates
coordinates = {
    "lat":34.00906474557528,
    "lon": -118.4984580927553
}

* Request from the endpoint.

In [3]:
# Build the url
url= f"https://api.electricitymap.org/v3/carbon-intensity/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"

# Print the endpoint
print("Endpoint: " + str(url))

Endpoint: https://api.electricitymap.org/v3/carbon-intensity/latest?lat=34.00906474557528&lon=-118.4984580927553


* Prepare the get request.

In [4]:
import requests
import helper

In [5]:
request = requests.get(
    url,
    headers={"auth-token": helper.load_emaps_api_key()})

In [6]:
# This byte format is more compact
request.content
type(request.content)

bytes

> Note: This byte format is more compact and often not human-readable. Additionally, it is not possible to access the fields within this byte data as we would access with key-value pairs in a dictionary. This is why we use json.loads()

In [7]:
import json

In [8]:
json.loads(request.content)

{'zone': 'US-CAL-LDWP',
 'carbonIntensity': 700,
 'datetime': '2024-07-22T15:00:00.000Z',
 'updatedAt': '2024-07-22T14:49:34.787Z',
 'createdAt': '2024-07-19T15:46:30.525Z',
 'emissionFactorType': 'lifecycle',
 'isEstimated': True,
 'estimationMethod': 'TIME_SLICER_AVERAGE'}

* Use the live power breakdown endpoint.

In [9]:
# Build the url
url = f"https://api.electricitymap.org/v3/power-breakdown/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"

In [10]:
print(url)

https://api.electricitymap.org/v3/power-breakdown/latest?lat=34.00906474557528&lon=-118.4984580927553


In [11]:
request = requests.get(
    url,
    headers={"auth-token": helper.load_emaps_api_key()})

In [12]:
power_breakdown = json.loads(request.content)

# Print the content
power_breakdown

{'zone': 'US-CAL-LDWP',
 'datetime': '2024-07-22T15:00:00.000Z',
 'updatedAt': '2024-07-22T14:49:34.787Z',
 'createdAt': '2024-07-19T15:46:30.525Z',
 'powerConsumptionBreakdown': {'nuclear': 0,
  'geothermal': 0,
  'biomass': 0,
  'coal': 0,
  'wind': 0,
  'solar': 0,
  'hydro': 0,
  'gas': 0,
  'oil': 0,
  'unknown': 219,
  'hydro discharge': 0,
  'battery discharge': 0},
 'powerProductionBreakdown': {'nuclear': None,
  'geothermal': None,
  'biomass': None,
  'coal': None,
  'wind': None,
  'solar': None,
  'hydro': None,
  'gas': None,
  'oil': None,
  'unknown': 219,
  'hydro discharge': None,
  'battery discharge': None},
 'powerImportBreakdown': {},
 'powerExportBreakdown': {},
 'fossilFreePercentage': 0,
 'renewablePercentage': 0,
 'powerConsumptionTotal': 219,
 'powerProductionTotal': 219,
 'powerImportTotal': None,
 'powerExportTotal': None,
 'isEstimated': True,
 'estimationMethod': 'TIME_SLICER_AVERAGE'}

* Print some specific values.

In [13]:
power_breakdown['renewablePercentage']

0

In [14]:
power_breakdown['fossilFreePercentage']

0

In [15]:
# Power Consumption Breakdown in MegaWatts
power_breakdown['powerConsumptionBreakdown']

{'nuclear': 0,
 'geothermal': 0,
 'biomass': 0,
 'coal': 0,
 'wind': 0,
 'solar': 0,
 'hydro': 0,
 'gas': 0,
 'oil': 0,
 'unknown': 219,
 'hydro discharge': 0,
 'battery discharge': 0}

* Do some math to understand better the values above.

In [16]:
import numpy as np

In [17]:
total_consumption = power_breakdown['powerConsumptionTotal']

In [18]:
total_consumption

219

In [19]:
consumption_percent = {
    k: np.round((v/total_consumption) * 100)
    for k,v
    in power_breakdown['powerConsumptionBreakdown'].items()}
consumption_percent

{'nuclear': 0.0,
 'geothermal': 0.0,
 'biomass': 0.0,
 'coal': 0.0,
 'wind': 0.0,
 'solar': 0.0,
 'hydro': 0.0,
 'gas': 0.0,
 'oil': 0.0,
 'unknown': 100.0,
 'hydro discharge': 0.0,
 'battery discharge': 0.0}

#### Helper function for the power_stats

In [20]:
import helper, requests, json, numpy as np
def power_stats(lat,lon, api_key=helper.load_emaps_api_key()):
    coordinates = { "lat": lat, "lon": lon }

    url_intensity = f"https://api.electricitymap.org/v3/carbon-intensity/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"
    request_intensity = requests.get(url_intensity, headers={"auth-token": api_key})
    intensity = json.loads(request_intensity.content)

    url_breakdown = f"https://api.electricitymap.org/v3/power-breakdown/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"
    request_breakdown = requests.get(url_breakdown, headers={"auth-token": api_key})
    breakdown = json.loads(request_breakdown.content)

    breakdown_abridged = {
        'renewablePercentage': breakdown['renewablePercentage'],
        'fossilFreePercentage': breakdown['fossilFreePercentage'],
        'powerConsumptionBreakdown': breakdown['powerConsumptionBreakdown'],
        'consumption_percent': {
            k: np.round((v/breakdown['powerConsumptionTotal']) * 100) 
            for k, v 
            in breakdown['powerConsumptionBreakdown'].items()
        },
    }
    
    return intensity, breakdown_abridged

In [21]:
# Coordinates from a landmark in Taiwan, shown by the instructor in the explanation
intensity, breakdown = power_stats(
    lat=25.0356575108668,
    lon=121.52010809479746)

In [22]:
intensity

{'zone': 'TW',
 'carbonIntensity': 570,
 'datetime': '2024-07-22T15:00:00.000Z',
 'updatedAt': '2024-07-22T14:49:34.787Z',
 'createdAt': '2024-07-19T15:46:17.152Z',
 'emissionFactorType': 'lifecycle',
 'isEstimated': True,
 'estimationMethod': 'TIME_SLICER_AVERAGE'}

In [23]:
breakdown

{'renewablePercentage': 5,
 'fossilFreePercentage': 9,
 'powerConsumptionBreakdown': {'nuclear': 1637,
  'geothermal': 2,
  'biomass': 20,
  'coal': 11426,
  'wind': 401,
  'solar': 0,
  'hydro': 1154,
  'gas': 16907,
  'oil': 933,
  'unknown': 1386,
  'hydro discharge': 0,
  'battery discharge': 0},
 'consumption_percent': {'nuclear': 5.0,
  'geothermal': 0.0,
  'biomass': 0.0,
  'coal': 34.0,
  'wind': 1.0,
  'solar': 0.0,
  'hydro': 3.0,
  'gas': 50.0,
  'oil': 3.0,
  'unknown': 4.0,
  'hydro discharge': 0.0,
  'battery discharge': 0.0}}

### Do it yourself! 
* Get coordinates from a location you want to retrieve the information we got before!